In [1]:
import numpy as np

In [389]:
class GridWorld(object):
    def __init__(self):
        self.grid = np.array([[0,0,0,0,0,0,0,0,0],
                              [0,1,1,1,1,2,0,0,0],
                              [0,2,2,2,0,2,2,0,0],
                              [0,0,0,0,0,0,0,2,0],
                              [0,0,2,2,2,2,0,2,0],
                              [0,0,0,0,0,0,0,2,0],
                              [0,2,2,2,0,2,2,0,0],
                              [0,0,0,0,0,0,0,0,3],
                              [0,1,1,1,2,1,1,0,2]]) 
        self.rewards = {0:-1, 1:5, 2:-20, 3:100}
        self.actions = np.array([[1, 0.125],[2, 0.125], [3, 0.625],[4, 0.125]]) # up, right, down, left
        self.transition_probability = np.array([0.2,0.6,0.2]) # left diagonal, desired direction, right diagonal
        self.gama = 0.9
        self.threshold = 0.01
        self.n=9
        self.a=4
        self.states = n*n

    def get_next_state(self,row,col,a):
        # going up
        if a == 1:
            # if state is in row 0 it stays in same row
            if row == 0:
                r = row
                c = col
                reward = -10
            else:
                # if next state is not reachable
                reward = self.rewards[grid[row-1,col]]
                if grid[row-1,col] == 2:
                    r = row
                    c = col
                # if next state is reachable and no obstacle
                else:
                    r = row-1
                    c = col
        # go right
        elif a == 2:
            # if state is in last column it stays in same column
            if col == 8:
                r = row
                c = col
                reward = -10
            else:
                # if next state is not reachable
                reward = self.rewards[grid[row,col+1]]
                if grid[row,col+1] == 2:
                    r = row
                    c = col
                # if next state is reachable and no obstacle
                else:
                    r = row
                    c = col+1
        # go down
        elif a == 3:
            # if state is in row 8 it stays in same row
            if row == 8:
                r = row
                c = col
                reward = -10
            else:
                # if next state is not reachable
                reward = self.rewards[grid[row+1,col]]
                if grid[row+1,col] == 2:
                    r = row
                    c = col
                # if next state is reachable and no obstacle
                else:
                    r = row+1
                    c = col
        # go left
        elif a == 4:
            # if state is in first column it stays in same column
            if col == 0:
                r = row
                c = col
                reward = -10
            else:
                # if next state is not reachable
                reward = self.rewards[grid[row,col-1]]
                if grid[row,col-1] == 2:
                    r = row
                    c = col
                # if next state is reachable and no obstacle
                else:
                    r = row
                    c = col-1
        return r,c,reward
    
    def get_next_non_determinstic_state(self,row,col,a):
        self.possible_states = np.zeros((3,4))
        if a == 1:
            for i in range(3):
                # Diagonaly left
                if i == 0:
                    if row == 0:
                        r = row
                        c = col
                        reward = -10
                    else:
                        if col == 0:
                            r = row
                            c = col
                            reward = -10
                        else:
                            reward = self.rewards[grid[row-1,col-1]]
                            if grid[row-1,col] == 2:
                                r = row
                                c = col
                            # if next state is reachable and no obstacle
                            else:
                                r = row-1
                                c = col-1
                    prob = self.transition_probability[i]
                # Desired direction
                if i == 1:
                    if row == 0:
                        r = row
                        c = col
                        reward = -10
                    else:
                        reward = self.rewards[grid[row-1,col]]
                        if grid[row-1,col] == 2:
                            r = row
                            c = col
                        # if next state is reachable and no obstacle
                        else:
                            r = row-1
                            c = col
                    prob = self.transition_probability[i]
                # Diagonally right
                if i == 2:
                    if row == 0:
                        r = row
                        c = col
                        reward = -10
                    else:
                        if col == 8:
                            r = row
                            c = col
                            reward = -10
                        else:
                            reward = self.rewards[grid[row-1,col+1]]
                            if grid[row-1,col+1] == 2:
                                r = row
                                c = col
                            # if next state is reachable and no obstacle
                            else:
                                r = row-1
                                c = col+1
                    prob = self.transition_probability[i]
                self.possible_states[i,:] = np.array([r,c,reward,prob])
        if a == 2:
            for i in range(3):
                # Diagonaly left
                if i == 0:
                    if col == 8:
                        r = row
                        c = col
                        reward = -10
                    else:
                        if row == 0:
                            r = row
                            c = col
                            reward = -10
                        else:
                            reward = self.rewards[grid[row-1,col+1]]
                            if grid[row-1,col+1] == 2:
                                r = row
                                c = col
                            # if next state is reachable and no obstacle
                            else:
                                r = row-1
                                c = col+1
                    prob = self.transition_probability[i]
                # Desired direction
                if i == 1:
                    if col == 8:
                        r = row
                        c = col
                        reward = -10
                    else:
                        reward = self.rewards[grid[row,col+1]]
                        if grid[row,col+1] == 2:
                            r = row
                            c = col
                        # if next state is reachable and no obstacle
                        else:
                            r = row
                            c = col+1
                    prob = self.transition_probability[i]
                # Diagonally right
                if i == 2:
                    if col == 8:
                        r = row
                        c = col
                        reward = -10
                    else:
                        if row == 8:
                            r = row
                            c = col
                            reward = -10
                        else:
                            reward = self.rewards[grid[row+1,col+1]]
                            if grid[row+1,col+1] == 2:
                                r = row
                                c = col
                            # if next state is reachable and no obstacle
                            else:
                                r = row+1
                                c = col+1
                    prob = self.transition_probability[i]
                self.possible_states[i,:] = np.array([r,c,reward,prob])
        if a == 3:
            for i in range(3):
                # Diagonaly left
                if i == 0:
                    if row == 8:
                        r = row
                        c = col
                        reward = -10
                    else:
                        if col == 8:
                            r = row
                            c = col
                            reward = -10
                        else:
                            reward = self.rewards[grid[row+1,col+1]]
                            if grid[row+1,col+1] == 2:
                                r = row
                                c = col
                            # if next state is reachable and no obstacle
                            else:
                                r = row+1
                                c = col+1
                    prob = self.transition_probability[i]
                # Desired direction
                if i == 1:
                    if row == 8:
                        r = row
                        c = col
                        reward = -10
                    else:
                        reward = self.rewards[grid[row+1,col]]
                        if grid[row+1,col] == 2:
                            r = row
                            c = col
                        # if next state is reachable and no obstacle
                        else:
                            r = row+1
                            c = col
                    prob = self.transition_probability[i]
                # Diagonally right
                if i == 2:
                    if row == 8:
                        r = row
                        c = col
                        reward = -10
                    else:
                        if col == 0:
                            r = row
                            c = col
                            reward = -10
                        else:
                            reward = self.rewards[grid[row+1,col-1]]
                            if grid[row+1,col-1] == 2:
                                r = row
                                c = col
                            # if next state is reachable and no obstacle
                            else:
                                r = row+1
                                c = col-1
                    prob = self.transition_probability[i]
                self.possible_states[i,:] = np.array([r,c,reward,prob])
        if a == 4:
            for i in range(3):
                # Diagonaly left
                if i == 0:
                    if col == 0:
                        r = row
                        c = col
                        reward = -10
                    else:
                        if row == 8:
                            r = row
                            c = col
                            reward = -10
                        else:
                            reward = self.rewards[grid[row+1,col-1]]
                            if grid[row+1,col-1] == 2:
                                r = row
                                c = col
                            # if next state is reachable and no obstacle
                            else:
                                r = row+1
                                c = col-1
                    prob = self.transition_probability[i]
                # Desired direction
                if i == 1:
                    if col == 0:
                        r = row
                        c = col
                        reward = -10
                    else:
                        reward = self.rewards[grid[row,col-1]]
                        if grid[row,col-1] == 2:
                            r = row
                            c = col
                        # if next state is reachable and no obstacle
                        else:
                            r = row
                            c = col-1
                    prob = self.transition_probability[i]
                # Diagonally right
                if i == 2:
                    if col == 0:
                        r = row
                        c = col
                        reward = -10
                    else:
                        if row == 0:
                            r = row
                            c = col
                            reward = -10
                        else:
                            reward = self.rewards[grid[row-1,col-1]]
                            if grid[row-1,col-1] == 2:
                                r = row
                                c = col
                            # if next state is reachable and no obstacle
                            else:
                                r = row-1
                                c = col-1
                    prob = self.transition_probability[i]
                self.possible_states[i,:] = np.array([r,c,reward,prob])
        return self.possible_states
        
    def get_value(self,row,col):
        index = row*9 + col
        return self.V[index]
    
    def get_next_state_value(self, row, col):
        A_v = np.zeros(self.a)
        for j in range(1,5): # for each action
            r,c,reward = self.get_next_state(row,col,j) # get next state and reward
            next_s = r*9 + c
            next_v = self.get_value(r,c) # get value from nest state
            prob = self.actions[j-1,1] # probability of action
            A_v[j-1]  += prob*(reward + gama*next_v)
        return A_v
    
    def get_next_non_deterministic_state_value(self, row, col):
        A_v = np.zeros(self.a)
        for j in range(1,5): # for each action
            possible_states = self.get_next_non_determinstic_state(row,col,j) # get next state and reward
            for s in range(3):
                next_s = possible_states[s,0]*9 + possible_states[s,1]
                next_v = self.get_value(possible_states[s,0],possible_states[s,1]) # get value from nest state
                prob = self.actions[j-1,1] # probability of action
                A_v[j-1]  +=  possible_states[s,3]*prob*(possible_states[s,2] + gama*next_v)
        return A_v

    def policy_evaluation(self):        
        self.V = np.zeros(self.states)
        count = 0
        iterate = True
        while iterate:
        #     print(count)
            delta = 0
            for i in range(self.states):
                row = int(i / n)
                col = i % n
                s = grid[row,col]
                v = 0
                if s != 2 and s != 3:
                    for j in range(1,5): # for each action
                        r,c,reward = self.get_next_state(row,col,j) # get next state and reward
                        next_s = r*9 + c
                        next_v = self.get_value(r,c) # get value from nest state
                        prob = self.actions[j-1,1] # probability of action
                        v  += prob*(reward + gama*next_v)
        #                 print(" row: " + str(row) + " col: " + str(col))
        #                 print("a: " + str(j) + " r: " + str(r) + " c: " + str(c) + " nest state: " + str(next_s) + " reward: " + str(reward) + " prob: " + str(prob) + " v: " + str(prob*(reward + gama*next_v)))
                    delta = max(delta, np.abs(v - self.V[i]))
                self.V[i] = v
            count += 1
            if delta < self.threshold:
                break
        return self.V, delta, count

    def value_iteration(self):        
        self.V = np.zeros(self.states)
        self.policy = np.zeros((self.states,self.a))
        count = 0
        iterate = True
        while iterate:
        #     print(count)
            delta = 0
            for i in range(self.states):
                row = int(i / n)
                col = i % n
                s = grid[row,col]
                best_v = 0
                if s != 2 and s != 3:
                    values = self.get_next_state_value(row,col)
                    best_v = max(values)
                    delta = max(delta, np.abs(best_v- self.V[i]))
                self.V[i] = best_v
            count += 1
            if delta < self.threshold:
                break
        
        for i in range(self.states):
            row = int(i / n)
            col = i % n
            values = self.get_next_state_value(row,col)
            best_v = np.argmax(values)
            self.policy[i,best_v] = 1
        # Deterministic policy     
        return self.V, delta, count, self.policy

    def non_deterministic_value_iteration(self):        
        self.V = np.zeros(self.states)
        self.policy = np.zeros((self.states,self.a))
        count = 0
        iterate = True
        while iterate:
        #     print(count)
            delta = 0
            for i in range(self.states):
                row = int(i / n)
                col = i % n
                s = grid[row,col]
                best_v = 0
                if s != 2 and s != 3:
                    values = self.get_next_non_deterministic_state_value(row,col)
                    best_v = max(values)
                    delta = max(delta, np.abs(best_v- self.V[i]))
                self.V[i] = best_v
            count += 1
            if delta < self.threshold:
                break
        
        for i in range(self.states):
            row = int(i / n)
            col = i % n
            values = self.get_next_state_value(row,col)
            best_v = np.argmax(values)
            self.policy[i,best_v] = 1
        # Deterministic policy     
        return self.V, delta, count, self.policy

In [390]:
grid_world = GridWorld()

In [391]:

V,delta, count = grid_world.policy_evaluation()
V.shape = (9,9)
print(delta)
print(count)
print(V)

0.00898890227799
57
[[ -50.08612369  -64.53334339  -72.59725479  -76.60701104  -80.75454646
   -99.117781    -81.17306149  -50.62544565  -20.11697798]
 [ -50.33345118  -80.18134925  -89.2138516   -93.09876195  -95.1679835
     0.          -94.34828439  -55.84078722  -11.81449416]
 [ -50.92566932    0.            0.            0.         -110.72720052
     0.            0.          -66.13771494    0.32820054]
 [ -52.10576651  -76.25650885 -118.69214086 -131.13425753 -124.64722049
  -128.48502285 -111.03299206    0.           19.88560342]
 [ -53.00511534  -80.16484961    0.            0.            0.            0.
  -117.06883055    0.           35.33180675]
 [ -53.40611199  -94.63685456 -120.05144636 -116.66504507  -81.46454262
  -119.71828251 -128.87486324    0.           52.70177889]
 [ -50.96431816    0.            0.            0.          -75.26272895
     0.            0.          -14.73982542   73.5445146 ]
 [ -51.5435615   -46.30713212  -46.57285074  -55.2849902   -77.18601655


In [392]:
V, delta, count, policy = grid_world.value_iteration()
V.shape = (9,9)
print(delta)
print(count)
print(V)
print("\n0 = up, 1 = right, 2 = down, 3 = left \n")
print(np.reshape(np.argmax(policy,axis=1),(9,9)))

1.6074364062e-12
9
[[  2.71126761e-01   3.52112676e+00   3.52112676e+00   3.52112676e+00
    3.52112676e+00   2.71126761e-01  -9.44982394e-02  -5.78990933e-02
    5.96452504e-01]
 [  7.04225352e-01   7.04225352e-01   7.04225352e-01   7.04225352e-01
    7.04225352e-01   0.00000000e+00  -1.11579819e-01   1.19290501e-01
    2.17147112e+00]
 [ -4.57746479e-02   0.00000000e+00   0.00000000e+00   0.00000000e+00
    7.04225352e-01   0.00000000e+00   0.00000000e+00   4.34294224e-01
    4.97150421e+00]
 [ -1.30149648e-01  -1.39641835e-01  -1.39641835e-01  -1.30149648e-01
   -4.57746479e-02  -1.30149648e-01  -1.39641835e-01   0.00000000e+00
    9.94934082e+00]
 [ -1.39641835e-01  -1.40709706e-01   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00  -1.40709706e-01   0.00000000e+00
    1.87988281e+01]
 [ -1.40709706e-01  -1.40829842e-01  -1.40843357e-01  -1.40844878e-01
   -1.40844878e-01  -1.40843357e-01  -1.40829842e-01   0.00000000e+00
    3.45312500e+01]
 [ -1.40829842e-01   

In [393]:
V, delta, count, policy = grid_world.non_deterministic_value_iteration()
V.shape = (9,9)
print(delta)
print(count)
print(V)
print("\n0 = up, 1 = right, 2 = down, 3 = left \n")
print(np.reshape(np.argmax(policy,axis=1),(9,9)))

0.00868604831232
8
[[ -3.44243877e-02   2.42739026e+00   3.24868693e+00   3.25252722e+00
    2.74076201e-02  -2.03966779e-01  -3.88366809e-01  -3.88262459e-01
   -3.88366809e-01]
 [ -8.99025690e-02   1.11169750e-01   2.22084511e-01   3.22016055e-01
   -5.45573757e-02   0.00000000e+00  -1.47190863e-01  -1.52037335e-01
   -1.47190863e-01]
 [ -2.08252792e-01   0.00000000e+00   0.00000000e+00   0.00000000e+00
    3.64474515e-03   0.00000000e+00   0.00000000e+00  -1.41886109e-01
   -3.71719922e-01]
 [ -6.29541091e-01  -1.75401765e-01  -6.29541091e-01  -6.89650886e-01
   -1.09949256e+00  -6.89650886e-01  -1.17439941e+00   0.00000000e+00
   -3.86990826e-01]
 [ -1.43202384e-01  -1.57191006e-01   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00  -7.10781693e-01   0.00000000e+00
    1.55768724e+00]
 [ -3.71563129e-01  -1.62020042e-01  -6.28617034e-01  -7.00388340e-01
   -1.17515834e+00  -7.00388340e-01  -1.14882636e+00   0.00000000e+00
    1.58051621e+01]
 [ -3.87443440e-01   

/home/rob/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:342: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
